In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report| grep "Live datanodes"


import urllib.request
urllib.request.urlretrieve("https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv", "hdma-wi-2021.csv")

import pyarrow.csv as pv
import pyarrow.parquet as pq

# df = pd.read_csv("hdma-wi-2021.csv", low_memory=False)

# table = pa.Table.from_pandas(df)
csv_reader = pv.read_csv("hdma-wi-2021.csv")
pq.write_table(csv_reader, "hdma-wi-2021.parquet")

! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp /nb/hdma-wi-2021.parquet hdfs://boss:9000/single.parquet
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp /nb/hdma-wi-2021.parquet hdfs://boss:9000/double.parquet

Live datanodes (2):


In [3]:
#q2
! hdfs dfs -du -h hdfs://boss:9000/

15.9 M  31.7 M  hdfs://boss:9000/double.parquet
15.9 M  15.9 M  hdfs://boss:9000/single.parquet


In [4]:
#q3
import requests

r= requests.get('http://boss:9870/webhdfs/v1/single.parquet?op=GETFILESTATUS')

# print(r.json())
r.json()

{'FileStatus': {'accessTime': 1729911971265,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 16642976,
  'modificationTime': 1729911972517,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [5]:
#q4
r= requests.get('http://boss:9870/webhdfs/v1/single.parquet?op=OPEN&offset=0&noredirect=true')

# print(r.json())
r.json()

{'Location': 'http://2c60d8032db3:9864/webhdfs/v1/single.parquet?op=OPEN&namenoderpcaddress=boss:9000&offset=0'}

In [6]:
#q5
from collections import defaultdict

r= requests.get("http://boss:9870/webhdfs/v1/single.parquet?op=GETFILEBLOCKLOCATIONS")


dict= {}
for topologyPath in r.json()["BlockLocations"]["BlockLocation"]:
    if (topologyPath["hosts"][0] not in dict):
        dict[topologyPath["hosts"][0]]= 1
    else:
        dict[topologyPath["hosts"][0]]+=1

#dictionary file
# print(dict)
dict

{'2c60d8032db3': 9, 'e18610104cd6': 7}

In [8]:
#q6
r= requests.get("http://boss:9870/webhdfs/v1/double.parquet?op=GETFILEBLOCKLOCATIONS")
# print(r.json())


dict = {}

for blk in r.json()["BlockLocations"]["BlockLocation"]:

    for host in blk["hosts"]:

        dict[host] = dict.get(host, 0) + 1

dict


# dict= {}
# for topologyPath in r.json()["BlockLocations"]["BlockLocation"]:
#     if (topologyPath["hosts"][0] not in dict):
#         dict[topologyPath["hosts"][0]]= 1
#     else:
#         dict[topologyPath["hosts"][0]]+=1

# print(dict)
# dict

{'e18610104cd6': 16, '2c60d8032db3': 16}

In [9]:
#q7
import pyarrow as pa
import pyarrow.fs
import time

hdfs = pa.fs.HadoopFileSystem("boss", 9000)


startTime = time.time()



with hdfs.open_input_file("/single.parquet") as f:
    f
    t = pa.parquet.read_table(f)


loanCol = t['loan_amount']

average = loanCol.to_pandas().mean()

endTime = time.time()

timeCost = endTime - startTime

# print(average)
average
# print(timeCost)
# timeCost

2024-10-26 03:07:17,882 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


np.float64(204961.21752386744)

In [10]:
#q8
startTime = time.time()
with hdfs.open_input_file("/single.parquet") as f:
    t = pa.parquet.read_table(f)
loanCol = t['loan_amount']
average = loanCol.to_pandas().mean()
endTime = time.time()
timeCost1 = endTime - startTime




startTime2 = time.time()

with hdfs.open_input_file("/single.parquet") as f:
    loanCol = pa.parquet.read_table(f, columns=["loan_amount"])
    
average = loanCol.to_pandas().mean()

endTime2 = time.time()

timeCost2 = endTime2 - startTime2

diff= timeCost1/timeCost2
# print(average)
# print(timeCost)
# print(diff)
diff

19.89287155011037